### Imports and setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install -U segmentation-models

     |████████████████████████████████| 50 kB 5.8 MB/s 


In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import segmentation_models as sm
import os
import matplotlib.image as mpimg
from PIL import Image
import cv2

Segmentation Models: using `keras` framework.


In [ ]:
!unzip "/content/drive/My Drive/ml/data.zip" -d "/content"

In [ ]:
!unzip "/content/drive/My Drive/ml/images.zip" -d "/content"

### Extracting data and masks

In [ ]:
def extract_data(folderpath):
    """ (ETH) Extract the images into a 4D tensor [image index, y, x, channels].
    Values are rescaled from [0, 255] down to [-0.5, 0.5].

    Parameters
    ----------
    filename: string
        The name of the image file
    num_images: int
        The number of images that should be extracted

    Returns
    -------
    data: ndarray
        A numpy array containting the images
    """
    files = os.listdir(folderpath)
    n = len(files)
    imgs = [(mpimg.imread(folderpath+files[i])) for i in range(n)]
    data = np.asarray(imgs)
    return data

def extract_data_test(folderpath):
    """ (ETH) Extract the images into a 4D tensor [image index, y, x, channels].
    Values are rescaled from [0, 255] down to [-0.5, 0.5].

    Parameters
    ----------
    filename: string
        The name of the image file
    num_images: int
        The number of images that should be extracted

    Returns
    -------
    data: ndarray
        A numpy array containting the images
    """
    imgs=[]
    for i in range(1,51):
      img = mpimg.imread(folderpath+'test_%d.png'%i)
      imgs.append(img)
    data = np.asarray(imgs)
    return data


def extract_labels(folderpath):
    """ (ETH) Extract the labels into a 1-hot matrix [image index, label index].
    
    Parameters
    ----------
    filename: string
        The name of the image file
    num_images: int
        The number of images
    
    Returns
    --------
    labels: ndarray
        1-hot matrix [image index, label index]
    """
    gt_imgs = []
    files = os.listdir(folderpath)
    n = len(files)
    for i in range(n):
        img = mpimg.imread(folderpath+files[i])
        try:
            gt_imgs.append(img[:,:,0])
        except:
            gt_imgs.append(img)

    return np.asarray(gt_imgs)

In [ ]:


train_data_path = '/content/data_volt/training/images/'
train_labels_path = '/content/data_volt/training/groundtruth/'
#val_data_path = '/content/training/images/'
#val_labels_path = '/content/training/groundtruth/10-split/'

x_train = extract_data(train_data_path)
#x_val = extract_data(val_data_path)
y_train = extract_labels(train_labels_path)
#y_val = extract_labels(val_labels_path)

In [ ]:
x_val = x_train[1360:]
y_val = y_train[1360:]
x_train = x_train[0:1360]
y_train = y_train[0:1360]

In [ ]:
x_train.shape

(1360, 256, 256, 3)

### Model time

In [ ]:
sm.set_framework('tf.keras')
import math
keras.backend.clear_session()
BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)
from sklearn.model_selection import train_test_split

# preprocess input
x_train = preprocess_input(x_train)
x_val = preprocess_input(x_val)


In [ ]:

# define model
model = sm.Unet(BACKBONE, encoder_weights='imagenet', input_shape=(256, 256, 3))

# adding a l2 kernel regularizer
sm.utils.set_regularization(model, kernel_regularizer=keras.regularizers.l2(1))
model.compile(
    'Adam',
    loss=sm.losses.bce_jaccard_loss,
    metrics=[sm.metrics.iou_score, sm.metrics.FScore(),'accuracy'],
)

# saving the best model
callbacks = [keras.callbacks.ModelCheckpoint("/content/drive/MyDrive/Models/m5.h5", save_best_only=True)]

# training
history = model.fit(x=x_train, y=y_train,
   epochs=50, batch_size=32,
   callbacks=callbacks,
   validation_data=(x_val,y_val)
)


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/50
43/43 [==============================] - 56s 1s/step - loss: 1.0729 - iou_score: 0.3130 - f1-score: 0.4672 - accuracy: 0.6731 - val_loss: 3.6639 - val_iou_score: 0.0045 - val_f1-score: 0.0090 - val_accuracy: 0.7711
Epoch 2/50
43/43 [==============================] - 49s 1s/step - loss: 0.7228 - iou_score: 0.5268 - f1-score: 0.6894 - accuracy: 0.7329 - val_loss: 3.3626 - val_iou_score: 0.0018 - val_f1-score: 0.0037 - val_accuracy: 0.7743
Epoch 3/50
43/43 [==============================] - 49s 1s/step - loss: 0.5972 - iou_score: 0.6135 - f1-score: 0.7601 - accuracy: 0.7441 - val_loss: 1.6000 - val_iou_score: 0.0400 - val_f1-score: 0.0768 - val_accuracy: 0.7743
Epoch 4/50
43/43 [==============================] - 44s 1s/step - loss: 0.5427 - iou_score: 0.6510 - f1-score: 0.7884 - accuracy: 0.7470 - val_loss: 1.6561 - val_iou_score: 0.0342 - val_f1-score: 0.0660 - val_accuracy: 0.7743
Epoch 5/50
43/43 [==============================] - 44s 1s/step - loss: 0.4979 - iou_score: 0.68

KeyboardInterrupt: ignored

### Testing and predicting

In [ ]:
x_val = extract_data('/content/images/')

In [ ]:
from keras.models import load_model

# loading the best model
model = load_model('/content/drive/MyDrive/Models/m5.h5', custom_objects
                   = {'binary_crossentropy_plus_jaccard_loss':sm.losses.bce_jaccard_loss, 
                      'iou_score': sm.metrics.iou_score, 'f1-score': sm.metrics.FScore()})

#from tensorflow.keras.utils import plot_model
#plot_model(model, to_file='model.png')

#test_images = extract_data_test('/content/testing/')



In [ ]:
# predicting on the validation set
y_pred = model.predict((x_val))
patch_size = 16

ValueError: ignored

Checking which threshold for converting patch to foregroud is best on the val set

In [ ]:
from sklearn.metrics import f1_score, accuracy_score
def patch_to_label(patch, thr):
    df = np.mean(patch)
    if df > thr:
        return 1
    else:
        return 0
thresholds = [0.01*i for i in range(1, 10)]
for thr in thresholds:
  highest=0
  y_pred_flattened = []
  for im in y_pred:
    for j in range(0, im.shape[1], patch_size):
          for i in range(0, im.shape[0], patch_size):
              patch = im[i:i + patch_size, j:j + patch_size]
              label = patch_to_label(patch, thr)
              y_pred_flattened.append(label)
  y_val_flattened = []
  for im in y_val:
    for j in range(0, im.shape[1], patch_size):
          for i in range(0, im.shape[0], patch_size):
              patch = im[i:i + patch_size, j:j + patch_size]
              label = patch_to_label(patch, thr)
              y_val_flattened.append(label)

  y_pred_flattened = np.array(y_pred_flattened)
  f1 = f1_score(y_val_flattened, y_pred_flattened)
  if f1>highest:
    foreground_threshold=thr
  acc = accuracy_score(y_val_flattened, y_pred_flattened)
  print(thr, f1, acc)

0.01 0.9865501650227979 0.9919002757352942
0.02 0.9870463297934415 0.9923483455882353
0.03 0.9868880257113001 0.9923598345588235
0.04 0.9871981785864073 0.9926355698529412
0.05 0.9864204017297822 0.9922794117647059
0.06 0.9871870849085522 0.9927964154411765
0.07 0.986010951544581 0.9922219669117647
0.08 0.9851793094810671 0.9918428308823529
0.09 0.985713081591369 0.9922104779411764


#Generating predictions

In [ ]:
BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)
#test_images = extract_data_test('/content/testing/')



In [ ]:
# predicting on 256x256 crops of the test images
imgs = preprocess_input(x_val)
resultat = []
for img in imgs:
    img1 = img[0:256,0:256,:]
    img2 = img[0:256,144:,:]
    img4 = img[144:,0:256,:]
    img5 = img[144:,144:,:]
    pred_1 = model.predict(np.expand_dims(img1, axis=0))[0]
    pred_2 = model.predict(np.expand_dims(img2, axis=0))[0]
    pred_4 = model.predict(np.expand_dims(img4, axis=0))[0]
    pred_5 = model.predict(np.expand_dims(img5, axis=0))[0]

    pred_2 = pred_2[:,112:,:]
    pred_4 = pred_4[112:,:,:]
    pred_5 = pred_5[112:,112:,:]
    a = np.hstack([pred_1,pred_2])
    b = np.hstack([pred_4,pred_5])
    d = np.vstack([a,b])
    resultat.append(d)

In [ ]:
d.shape

(400, 400, 1)

In [ ]:
def img_float_to_uint8(img):
    '''converts image array with floats to uint8
    
    parameters
    -----------
    img: ndarray
        image array
    
    returns
    -------
    rimg: ndarray
        converted array'''

    rimg = img - np.min(img)
    rimg = (rimg / np.max(rimg) * 255).round().astype(np.uint8)
    return rimg

# generating the prediction masks testset
for i in range(len(resultat)):
  img = resultat[i]
  w = img.shape[0]
  h = img.shape[1]
  gt_img_3c = np.zeros((w, h, 3), dtype=np.uint8)
  gt_img8 = img_float_to_uint8(img)          
  gt_img_3c[:, :, 0] = gt_img8[:,:,0]
  gt_img_3c[:, :, 1] = gt_img8[:,:,0]
  gt_img_3c[:, :, 2] = gt_img8[:,:,0]

  cv2.imwrite('/content/drive/MyDrive/Pred/test%d.png'%(i+1),gt_img_3c)


In [ ]:
# saving mask predictions on the validation set
for i in range(len(y_pred)):
  img = y_pred[i]
  w = img.shape[0]
  h = img.shape[1]
  gt_img_3c = np.zeros((w, h, 3), dtype=np.uint8)
  gt_img8 = img_float_to_uint8(img)          
  gt_img_3c[:, :, 0] = gt_img8[:,:,0]
  gt_img_3c[:, :, 1] = gt_img8[:,:,0]
  gt_img_3c[:, :, 2] = gt_img8[:,:,0]

  cv2.imwrite('/content/drive/MyDrive/masks/test%d.png'%(i+1),gt_img_3c)

In [ ]:
#!/usr/bin/env python3

import os
import numpy as np
import matplotlib.image as mpimg
import re

foreground_threshold = 0.04 # percentage of pixels > 1 required to assign a foreground label to a patch

# assign a label to a patch
def patch_to_label(patch):
    df = np.mean(patch)
    if df > foreground_threshold:
        return 1
    else:
        return 0


def mask_to_submission_strings(image_filename):
    """Reads a single image and outputs the strings that should go into the submission file"""
    img_number = int(re.search(r"\d+", image_filename).group(0))
    im = mpimg.imread(image_filename)
    patch_size = 16
    for j in range(0, im.shape[1], patch_size):
        for i in range(0, im.shape[0], patch_size):
            patch = im[i:i + patch_size, j:j + patch_size]
            label = patch_to_label(patch)
            yield("{:03d}_{}_{},{}".format(img_number, j, i, label))


def masks_to_submission(submission_filename, *image_filenames):
    """Converts images into a submission file"""
    with open(submission_filename, 'w') as f:
        f.write('id,prediction\n')
        for fn in image_filenames[0:]:
            f.writelines('{}\n'.format(s) for s in mask_to_submission_strings(fn))


# generating the prediction file for the test set
submission_filename = '/content/drive/MyDrive/Pred/m5.csv'
image_filenames = []
for i in range(1, 51):
    image_filename = '/content/drive/MyDrive/Pred/test%d.png' % i
    print(image_filename)
    image_filenames.append(image_filename)
masks_to_submission(submission_filename, *image_filenames)


/content/drive/MyDrive/Pred/test1.png
/content/drive/MyDrive/Pred/test2.png
/content/drive/MyDrive/Pred/test3.png
/content/drive/MyDrive/Pred/test4.png
/content/drive/MyDrive/Pred/test5.png
/content/drive/MyDrive/Pred/test6.png
/content/drive/MyDrive/Pred/test7.png
/content/drive/MyDrive/Pred/test8.png
/content/drive/MyDrive/Pred/test9.png
/content/drive/MyDrive/Pred/test10.png
/content/drive/MyDrive/Pred/test11.png
/content/drive/MyDrive/Pred/test12.png
/content/drive/MyDrive/Pred/test13.png
/content/drive/MyDrive/Pred/test14.png
/content/drive/MyDrive/Pred/test15.png
/content/drive/MyDrive/Pred/test16.png
/content/drive/MyDrive/Pred/test17.png
/content/drive/MyDrive/Pred/test18.png
/content/drive/MyDrive/Pred/test19.png
/content/drive/MyDrive/Pred/test20.png
/content/drive/MyDrive/Pred/test21.png
/content/drive/MyDrive/Pred/test22.png
/content/drive/MyDrive/Pred/test23.png
/content/drive/MyDrive/Pred/test24.png
/content/drive/MyDrive/Pred/test25.png
/content/drive/MyDrive/Pred/test26

In [ ]:
# generating the prediction file for the validation set
submission_filename = '/content/drive/MyDrive/masks/m1.csv'
image_filenames = []
for i in range(1, 341):
    image_filename = '/content/drive/MyDrive/masks/test%d.png' % i
    print(image_filename)
    image_filenames.append(image_filename)
masks_to_submission(submission_filename, *image_filenames)

/content/drive/MyDrive/masks/test1.png
/content/drive/MyDrive/masks/test2.png
/content/drive/MyDrive/masks/test3.png
/content/drive/MyDrive/masks/test4.png
/content/drive/MyDrive/masks/test5.png
/content/drive/MyDrive/masks/test6.png
/content/drive/MyDrive/masks/test7.png
/content/drive/MyDrive/masks/test8.png
/content/drive/MyDrive/masks/test9.png
/content/drive/MyDrive/masks/test10.png
/content/drive/MyDrive/masks/test11.png
/content/drive/MyDrive/masks/test12.png
/content/drive/MyDrive/masks/test13.png
/content/drive/MyDrive/masks/test14.png
/content/drive/MyDrive/masks/test15.png
/content/drive/MyDrive/masks/test16.png
/content/drive/MyDrive/masks/test17.png
/content/drive/MyDrive/masks/test18.png
/content/drive/MyDrive/masks/test19.png
/content/drive/MyDrive/masks/test20.png
/content/drive/MyDrive/masks/test21.png
/content/drive/MyDrive/masks/test22.png
/content/drive/MyDrive/masks/test23.png
/content/drive/MyDrive/masks/test24.png
/content/drive/MyDrive/masks/test25.png
/content/